In [2]:
# Reading all three csv files
import pandas as pd
data_grade_train = pd.read_excel('/content/drive/MyDrive/over_sampling_dataset/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.xlsx')

In [ ]:
data_grade_test = pd.read_csv('/content/drive/MyDrive/over_sampling_dataset/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv')

In [ ]:
data_grade_train = data_grade_train[['Image name','Retinopathy grade','Risk of macular edema ']]

In [ ]:
data_grade_test = data_grade_test[['Image name','Retinopathy grade','Risk of macular edema ']]

In [ ]:
print(data_grade_train)
print(data_grade_test)

In [ ]:
label_train = data_grade_train['Risk of macular edema ']
label_test = data_grade_test['Risk of macular edema '] 
print(label_train)
print(label_test)

from keras.utils.np_utils import to_categorical
label_train_cat = to_categorical(label_train)
label_test_cat = to_categorical(label_test)
print("label_train_cat.shape",label_train_cat.shape)
print("label_test_cat.shape",label_test_cat.shape)

In [ ]:
image_path_train = data_grade_train['Image name']
image_path_train
import numpy as np
import cv2
import os
from skimage import io
image_read_train = []
for imagepath in image_path_train:
  print(imagepath)
  image_path1_train = os.path.join('/content/drive/MyDrive/over_sampling_dataset/1. Original Images/a. Training Set 577',imagepath+'.jpg')
  #print(image_path1)
  image1_train = cv2.imread(image_path1_train)

  #median_train = cv2.medianBlur(image1_train,5)

  #Control Limited Adaptive Histogram Equlization (CLAHE)
  '''
  lab_img = cv2.cvtColor(median_train, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab_img) 
  clahe = cv2.createCLAHE(clipLimit = 3.0, tileGridSize = (8,8))
  clahe_img = clahe.apply(l)
  updated_lab_img = cv2.merge((clahe_img,a,b))
  CLAHE_img = cv2.cvtColor(updated_lab_img,cv2.COLOR_LAB2BGR)
  '''  
  green_channel = image1_train[:,:,1]
  # create empty image with same shape as that of src image
  green_img = np.zeros(image1_train.shape)
  #assign the green channel of src to empty image
  green_img[:,:,1] = green_channel
  
  image2_train = cv2.resize(green_img,(448,448))
  image3_train = image2_train.astype('float32')
  image4_train = image3_train/255
  image_read_train.append(image4_train)
image_array_train = np.asarray(image_read_train)

In [ ]:
image_path_test = data_grade_test['Image name']
image_path_test
image_read_test = []
for imagepath in image_path_test:
  print(imagepath)
  image_path1_test = os.path.join('/content/drive/MyDrive/over_sampling_dataset/1. Original Images/b. Testing Set 103',imagepath+'.jpg')
  #print(image_path1_test)
  image1_test = cv2.imread(image_path1_test)
  #median_test = cv2.medianBlur(image1_test,5)

  #Control Limited Adaptive Histogram Equlization (CLAHE)
  '''
  lab_img = cv2.cvtColor(median_test, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab_img)
  clahe = cv2.createCLAHE(clipLimit = 3.0, tileGridSize = (8,8))
  clahe_img = clahe.apply(l)
  updated_lab_img = cv2.merge((clahe_img,a,b))
  CLAHE_img = cv2.cvtColor(updated_lab_img,cv2.COLOR_LAB2BGR)
  '''
  
  green_channel = image1_test[:,:,1]

  # create empty image with same shape as that of src image
  green_img = np.zeros(image1_test.shape)

  #assign the green channel of src to empty image
  green_img[:,:,1] = green_channel
  print(green_img.shape)
  image2_test = cv2.resize(green_img,(448,448))
  image3_test = image2_test.astype('float32')
  image4_test = image3_test/255
  print(image4_test.shape)
  image_read_test.append(image4_test)
image_array_test = np.asarray(image_read_test)

In [ ]:
print(image_array_train.shape)

In [ ]:
print(image_array_train)

In [ ]:
print(image_array_test.shape)

In [ ]:
print(image_array_test)
#print(image_array_test.shape)

In [ ]:
print(image_array_train.shape,image_array_test.shape)

In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

In [ ]:
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Input, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization, concatenate, AveragePooling2D

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121
import tensorflow as tf

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout
import tensorflow.keras.layers as L


input_img = Input(shape=(448, 448, 3))

densenet121 = DenseNet121(input_shape=(448, 448,3),
                                             weights='imagenet',
                                             include_top=False)
x1 = densenet121.output
x2 = GlobalAveragePooling2D()(x1)
#model_concat = concatenate([x2, input_2], axis=-1,name="final_feature")

x22 = Dense(512,activation='relu')(x2)
x222 = Dense(256,activation='relu',name="final_feature")(x22)
x3 = Dropout(.5)(x222)
x4 = Dense(3,activation='softmax')(x3)
model=Model(inputs=densenet121.input,outputs=x4)



densenet121.summary()


In [ ]:
#Summary of model after adding droput and output layer
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=.0005),loss='categorical_crossentropy',
metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

es_densenet_121 = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=150)
mc_densenet_121 = ModelCheckpoint('/content/drive/MyDrive/models_saved/densenet121_256_features.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
hist_densenet121 = model.fit(image_array_train, label_train_cat, validation_data = (image_array_test, label_test_cat), 
           batch_size=16, epochs=150, verbose=1, callbacks=[es_densenet_121, mc_densenet_121])


In [ ]:
# load the saved model
saved_densenet = load_model('/content/drive/MyDrive/models_saved/densenet121_256_features.h5') 
# evaluate the model
_, train_acc_densenet = saved_densenet.evaluate(image_array_train, label_train_cat, verbose=0)
_, test_acc_densenet = saved_densenet.evaluate(image_array_test, label_test_cat, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc_densenet, test_acc_densenet))



In [ ]:
from sklearn import metrics
label_pred = saved_densenet.predict(image_array_test)  
#saved_densenet = load_model('best_model_glcm_densenet.h5') 


pred = []
for i in range(len(label_pred)):
    pred.append(np.argmax(label_pred[i]))

Y_test = np.argmax(label_test_cat, axis=1) 

#print(metrics.classification_report(Y_test, pred))
basic_classification_report=metrics.classification_report(Y_test, pred)
print(basic_classification_report)